In [1]:
import pandas as pd
import numpy as np
import altair as alt

from src.conf import settings

In [2]:
# Enable Altair to Serve Data remotely since we expect to have at least 8760 rows or more
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [3]:
year = 2019

df = pd.read_parquet(settings.DATA_DIR / f"processed/caiso/{year}.parquet").tz_convert(tz="US/Pacific")

column_map = zip(
    df.columns.tolist(), 
    df.columns.str.replace("\W+", "_").str.lower().tolist()
)

df = df.rename(columns=dict(column_map))

# Data Dictionary

- load (MW): Total demand across the CAISO for a given interval.
- solar (MW): Average interval Solar production
- wind (MW): Average interval Wind production
- net_load (MW): Load - solar - wind
- renewables (MW): Average interval production from solar, wind, biomass, biogas, geothermal and small hydropower
- nuclear (MW): Average nuclear production
- large_hydro (MW): Average large hydro production
- imports (MW): Imports coming into the ISO; note that exports are NOT deducted from imports.
- generation (MW): Total generation across all generator types
- thermal (MW): non-nuclear and non-geothermal thermal production
- load_less_generation_imports_ (MW): data validation column to ensure supply ~ demand
- wind_curtailment (MW): Curtailed wind in a given interval
- solar_curtailment (MW): Curtailed solar in a given interval


In [4]:
df.head()

,load,solar,wind,net_load,renewables,nuclear,large_hydro,imports,generation,thermal,load_less_generation_imports_,wind_curtailment,solar_curtailment
timestamp,,,,,,,,,,,,,
2019-01-01 00:00:00-08:00,22320.49429,0.0,2862.309099,19458.185191,4662.445970,2272.381180,2366.018830,6275.014973,16047.98977,6747.143790,-2.510453,NaN,NaN
2019-01-01 00:05:00-08:00,22295.40335,0.0,2915.095401,19380.307949,4715.330280,2272.392795,2355.716954,6314.684820,15980.83268,6637.392651,-0.114150,NaN,NaN
2019-01-01 00:10:00-08:00,22204.14444,0.0,2919.425381,19284.719059,4716.911237,2272.121178,2345.795928,6367.865026,15839.05150,6504.223157,-2.772086,NaN,NaN
2019-01-01 00:15:00-08:00,22114.41589,0.0,2901.973157,19212.442733,4700.156881,2272.328118,2354.491375,6342.362277,15768.76145,6441.785076,3.292163,NaN,NaN
2019-01-01 00:20:00-08:00,22035.12154,0.0,2874.352516,19160.769024,4674.126898,2271.824403,2346.957541,6321.318430,15712.61950,6419.710658,1.183610,NaN,NaN


Since our interval timeseries are not continuous and complete (i.e. significant irregular gaps exist across years), we must convert this to MWh to get additivity.
All columns are in MW at 5 minute intervals. MWh are calculated by taking

$MWh = MW  5[min]\frac{1[hour]}{60[min]}$

In [5]:
df_mwh = df * (5/60.)
df_mwh.head()

,load,solar,wind,net_load,renewables,nuclear,large_hydro,imports,generation,thermal,load_less_generation_imports_,wind_curtailment,solar_curtailment
timestamp,,,,,,,,,,,,,
2019-01-01 00:00:00-08:00,1860.041191,0.0,238.525758,1621.515433,388.537164,189.365098,197.168236,522.917914,1337.332481,562.261983,-0.209204,NaN,NaN
2019-01-01 00:05:00-08:00,1857.950279,0.0,242.924617,1615.025662,392.944190,189.366066,196.309746,526.223735,1331.736057,553.116054,-0.009512,NaN,NaN
2019-01-01 00:10:00-08:00,1850.345370,0.0,243.285448,1607.059922,393.075936,189.343431,195.482994,530.655419,1319.920958,542.018596,-0.231007,NaN,NaN
2019-01-01 00:15:00-08:00,1842.867991,0.0,241.831096,1601.036894,391.679740,189.360676,196.207615,528.530190,1314.063454,536.815423,0.274347,NaN,NaN
2019-01-01 00:20:00-08:00,1836.260128,0.0,239.529376,1596.730752,389.510575,189.318700,195.579795,526.776536,1309.384958,534.975888,0.098634,NaN,NaN


In [6]:
# Roll up to hourly 8760
hourly_mwh = df_mwh.groupby(
    by=pd.Grouper(freq="H")
)[["solar_curtailment", "solar", "net_load", "load", "generation", "renewables", "wind_curtailment"]].sum()

# Calculate some other potentially interesting metrics

# How much of our load in a given hour was fulfilled by solar generation?
hourly_mwh["pct_solar"] = hourly_mwh["solar"]/hourly_mwh["generation"]

# How much curtailment is occurring for each unit of raw solar generation?
hourly_mwh["curtailment_intensity"] = (hourly_mwh["solar_curtailment"].fillna(0)/hourly_mwh["solar"]).fillna(0)

# Calculate Pct of Total Solar Potential
hourly_mwh["pct_solar_potential"] = (hourly_mwh["solar_curtailment"]/(hourly_mwh["solar"] + hourly_mwh["solar_curtailment"])).fillna(0)

# Calculate Pct of Total Renewable Potential
hourly_mwh["pct_renewable_potential"] = (hourly_mwh["solar_curtailment"]/(
    hourly_mwh["renewables"] + hourly_mwh["solar_curtailment"] + hourly_mwh["wind_curtailment"])
).fillna(0)

In [7]:
basechart = alt.Chart(
    hourly_mwh.reset_index()
).mark_rect().encode(
    alt.X("hours(timestamp):T", title="Hour of Day"), 
    alt.Y("monthdate(timestamp):T", title="Date", sort="descending"),
).properties(
    width=200,
    height=1200
)
color_options = dict(
     
    sort="descending"
)

Below, we look at the annual curtailment.

In [8]:
alt.hconcat(
    basechart.encode(
        alt.Color(
            "solar_curtailment:Q", 
              scale=alt.Scale(type="pow", exponent=1/4, scheme="redgrey"), 
              sort="descending",
              title="Solar Curtailment (MWh)",
              legend=alt.Legend(orient="top"),
         )
    ),
    basechart.encode(
        alt.Color(
            "pct_solar_potential:Q", 
              scale=alt.Scale(type="pow", exponent=1/4, scheme="redgrey"),
              sort="descending",
              title="% Solar Potential",
              legend=alt.Legend(orient="top"),
        )
    ),
    basechart.encode(
        alt.Color(
            "curtailment_intensity:Q", 
            scale=alt.Scale(scheme="redgrey", type="pow", exponent=1/4, domain=[0,1]), 
            sort="descending", 
            title="MWh Cur. per MWh Prod.",
            legend=alt.Legend(orient="top"),
        )
    )
).resolve_scale(color="independent")

alt.HConcatChart(...)

Above, we see that curtailment is the most intense and most frequent

In [9]:
# Let's inspect our curtailed hours a bit more closely
curtailment_days = hourly_mwh.query("solar_curtailment > 0").reset_index()
alt.hconcat(
    alt.Chart(curtailment_days.sort_values("solar_curtailment", ascending=False).reset_index(drop=True).reset_index()).mark_line().encode(
        alt.Y("solar_curtailment", title="Solar Curtailment (MW)"),
        alt.X("index", title="rank")
    ),
    alt.Chart(curtailment_days).mark_point().encode(
        alt.X("load", title="Load (MW)"),
        alt.Y("solar_curtailment", title="Solar Curtailment (MW)")
    )
)

alt.HConcatChart(...)

Naively, there appears to be some kind of cut-off within the load relative to peak load with a higher class of curtailment events.  Looking at the "curtailed capacity" is also useful for classifying events.  A

In [10]:
alt.Chart(curtailment_days).mark_point().encode(
    alt.X("solar", title="Solar Production"),
    alt.Y("solar_curtailment", title="Solar Curtailment (MW)")
)

alt.Chart(...)

There exists a "curtailment boundary", where at each bin of production, there exists a maximum amount that can be curtailed. This probably represents the total solar potential for a given hour.